In [16]:
from google.colab import drive
drive.mount('/content/drive',force_remount='True')

Mounted at /content/drive


In [17]:
!pip install mlflow --quiet
!pip install pyngrok --quiet

In [18]:
import time

# 1. Tuer tous les processus MLflow et ngrok pour garantir une table rase
! pkill -f mlflow
! pkill -f ngrok

# 2. Lancer MLflow avec nohup pour le rendre robuste en arrière-plan.
# C'est l'option CRUCIALE : --host 0.0.0.0
# Nous redirigeons la sortie vers mlflow.log pour la vérification.
get_ipython().system_raw(
    "mlflow ui --host 0.0.0.0 --port 5000 "
    "--allowed-hosts='*' --cors-allowed-origins='*' "
    "--backend-store-uri file:///content/drive/MyDrive/mlruns "
    "--default-artifact-root file:///content/drive/MyDrive/mlartifacts &"
    "> mlflow.log 2>&1 &"
)


print("MLflow UI a été relancé sur 0.0.0.0. Attente de démarrage (10 secondes)...")
time.sleep(10) # Temps d'attente suffisant pour le démarrage

# 3. Vérification du journal (pour s'assurer que le serveur a démarré)
print("\n--- Journal de démarrage MLflow ---")
! tail -n 10 mlflow.log

MLflow UI a été relancé sur 0.0.0.0. Attente de démarrage (10 secondes)...

--- Journal de démarrage MLflow ---
Registry store URI not provided. Using backend store URI.
/usr/local/lib/python3.12/dist-packages/mlflow/server/handlers.py:259: FutureWarning: The filesystem tracking backend (e.g., './mlruns') will be deprecated in February 2026. Consider transitioning to a database backend (e.g., 'sqlite:///mlflow.db') to take advantage of the latest MLflow features. See https://github.com/mlflow/mlflow/issues/18534 for more details and migration guidance.
  return FileStore(store_uri, artifact_uri)
/usr/local/lib/python3.12/dist-packages/mlflow/server/handlers.py:288: FutureWarning: The filesystem model registry backend (e.g., './mlruns') will be deprecated in February 2026. Consider transitioning to a database backend (e.g., 'sqlite:///mlflow.db') to take advantage of the latest MLflow features. See https://github.com/mlflow/mlflow/issues/18534 for more details and migration guidance.
  

In [19]:
from pyngrok import ngrok, conf
from getpass import getpass
#36kqplEYfp2CyUaSPQb8uo8Zi3n_5RoQgrw5vWUmkzSJKkoy2

# 1. Configuration du jeton ngrok (assurez-vous que votre jeton est encore valable)
try:
    if not conf.get_default().auth_token:
        # Ceci est exécuté si le jeton n'est pas encore en mémoire
        ngrok.kill()
        NGROK_AUTH_TOKEN = getpass('Entrez le jeton d\'authentification ngrok: ')
        conf.get_default().auth_token = NGROK_AUTH_TOKEN
    else:
        # Ceci est exécuté si le jeton est déjà en mémoire
        ngrok.kill()
        print("Jeton ngrok déjà configuré. Connexion...")
except:
    # Cas de secours pour ressaisir le jeton
    ngrok.kill()
    NGROK_AUTH_TOKEN = getpass('Entrez le jeton d\'authentification ngrok: ')
    conf.get_default().auth_token = NGROK_AUTH_TOKEN

# 2. Ouvrir le tunnel HTTP vers le port 5000
ngrok_tunnel = ngrok.connect(addr="5000", proto="http", bind_tls=True)

# 3. Afficher l'URL publique
print("\n---")
print(f"MLflow Tracking UI est accessible via ce lien :\n{ngrok_tunnel.public_url}")
print("---")

Jeton ngrok déjà configuré. Connexion...

---
MLflow Tracking UI est accessible via ce lien :
https://interjectorily-accipitral-manuel.ngrok-free.dev
---


In [20]:
import mlflow
import mlflow.keras
import mlflow.sklearn

mlflow.set_tracking_uri("http://127.0.0.1:5000")  # ou l’URL ngrok
mlflow.set_experiment("Classification_Ecom_Store")

mlflow.keras.autolog()      # Track automatique Keras
mlflow.sklearn.autolog()    # Track sklearn

In [21]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.regularizers import l2

num_classes = 30  # nombre de classes dans ton dataset
input_shape = (128,128, 3)

# Charger MobileNetV2 pré-entraîné
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)

# Geler le backbone pour commencer
base_model.trainable = False

x = GlobalAveragePooling2D()(base_model.output)
x = Dense(512, activation='relu', kernel_regularizer=l2(1e-4))(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu', kernel_regularizer=l2(1e-4))(x)
x = Dropout(0.3)(x)
outputs = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=outputs)

model.compile(optimizer=tf.keras.optimizers.Adam(0.0001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [22]:
import numpy as np
import pickle
from sklearn.model_selection import train_test_split

# --- Charger CIFAR-100 ---
data = np.load("drive/MyDrive/cifar100.npz")
x_train, y_train = data['x_train'], data['y_train']
x_test, y_test   = data['x_test'], data['y_test']

# Flatten labels si nécessaire
y_train = y_train.flatten()
y_test  = y_test.flatten()

# Charger les noms des classes depuis le fichier meta
def load_cifar100_meta(path):
    with open(path, 'rb') as f:
        meta = pickle.load(f, encoding='latin1')
    return meta

meta = load_cifar100_meta("drive/MyDrive/meta")
class_names = meta['fine_label_names']

# --- Classes à garder ---
selected_classes = [
    'bottle','bowl','can','cup','plate',
    'apple','mushroom','orange','pear','sweet_pepper',
    'clock','keyboard','lamp','telephone','television',
    'bed','chair','couch','table','wardrobe',
    'bicycle','bus','motorcycle','pickup_truck','train',
    'lawn_mower','rocket','streetcar','tank','tractor'
]

# Convertir en indices CIFAR-100
selected_indices = [class_names.index(c) for c in selected_classes]

# --- Filtrer les données ---
mask_train = np.isin(y_train, selected_indices)
mask_test  = np.isin(y_test, selected_indices)

x_train = x_train[mask_train]
y_train = y_train[mask_train]

x_test  = x_test[mask_test]
y_test  = y_test[mask_test]

x_train = np.uint8(x_train*255)
x_test = np.uint8(x_test*255)


print("Train filtré :", x_train.shape, y_train.shape)
print("Test filtré  :", x_test.shape, y_test.shape)

# --- Remapper les labels de 0 à 29 ---
new_label_map = {old: i for i, old in enumerate(selected_indices)}
y_train = np.array([new_label_map[y] for y in y_train])
y_test  = np.array([new_label_map[y] for y in y_test])

print("Labels train :", np.unique(y_train))
print("Labels test  :", np.unique(y_test))

# --- Créer un ensemble de validation (10% du training) ---
x_train, x_val, y_train, y_val = train_test_split(
    x_train, y_train, test_size=0.1,
    random_state=42, stratify=y_train
)

import tensorflow as tf

x_train = tf.image.resize(
    x_train, (128, 128), method='bicubic'
).numpy().astype(np.uint8)

x_val = tf.image.resize(
    x_val, (128, 128), method='bicubic'
).numpy().astype(np.uint8)
x_test = tf.image.resize(
    x_test, (128, 128), method='bicubic'
).numpy().astype(np.uint8)

with mlflow.start_run(run_name="Dataset Details"):
  mlflow.log_param("train_size", len(x_train))
  mlflow.log_param("val_size", len(x_val))
  mlflow.log_param("test_size", len(x_test))
  mlflow.log_param("selected_classes", selected_classes)
mlflow.end_run()

print("Shapes après split :")
print("x_train:", x_train.shape, "y_train:", y_train.shape)
print("x_val:", x_val.shape, "y_val:", y_val.shape)
print("x_test:", x_test.shape, "y_test:", y_test.shape)


Exception ignored in: <function NpzFile.__del__ at 0x7fe08abe3a60>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/numpy/lib/_npyio_impl.py", line 226, in __del__
    self.close()
  File "/usr/local/lib/python3.12/dist-packages/numpy/lib/_npyio_impl.py", line 221, in close
    self.fid.close()
OSError: [Errno 107] Transport endpoint is not connected


Train filtré : (15000, 64, 64, 3) (15000,)
Test filtré  : (3000, 64, 64, 3) (3000,)
Labels train : [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]
Labels test  : [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]
🏃 View run Dataset Details at: http://127.0.0.1:5000/#/experiments/845622168192831652/runs/f492e15d6b08446c9a99c632e1cd8d42
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/845622168192831652
Shapes après split :
x_train: (13500, 128, 128, 3) y_train: (13500,)
x_val: (1500, 128, 128, 3) y_val: (1500,)
x_test: (3000, 128, 128, 3) y_test: (3000,)


In [23]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],   # variation de luminosité
    shear_range=0.1,                # décalage angulaire
    channel_shift_range=10          # modification légère des couleurs
)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

test_generator = test_datagen.flow(
    x_test,  # ou x_test_resized si tu as fait le resize en 128x128
    y_test,
    batch_size=32,
    shuffle=False
)


# Pas d'augmentation pour validation
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# Générateur pour l'entraînement
train_generator = train_datagen.flow(
    x_train, y_train, batch_size=32
)

# Générateur pour la validation
val_generator = val_datagen.flow(
    x_val, y_val,
    batch_size=32,
    shuffle=False
)


In [24]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

checkpoint_bf_ft = ModelCheckpoint(
    filepath="drive/MyDrive/Model/training_checkpoint_bf_ft.keras",
    save_weights_only=False,  # <--- important : sauver modèle + optimizer
    save_best_only=False,     # <--- pour avoir un checkpoint à chaque epoch
    save_freq="epoch",
    verbose=1
)

checkpoint_ft = ModelCheckpoint(
    filepath="drive/MyDrive/Model/training_checkpoint_ft.keras",
    save_weights_only=False,  # <--- important : sauver modèle + optimizer
    save_best_only=False,     # <--- pour avoir un checkpoint à chaque epoch
    save_freq="epoch",
    verbose=1
)

callbacks_bf_ft = [
    EarlyStopping(patience=5, restore_best_weights=True),
    ReduceLROnPlateau(factor=0.2, patience=2),
    checkpoint_bf_ft
]

callbacks_ft = [
    EarlyStopping(patience=5, restore_best_weights=True),
    ReduceLROnPlateau(factor=0.2, patience=2),
    checkpoint_ft
]



In [ ]:
with mlflow.start_run(run_name="Phase_1_Top_Classifier"):

    # paramètres manuels (facultatifs, mais recommandés)
    mlflow.log_param("input_shape", input_shape)
    mlflow.log_param("num_classes", num_classes)
    mlflow.log_param("augmentation", True)
    mlflow.log_param("epochs_phase1", 25)

    # Entraînement
    history = model.fit(
        train_generator,
        validation_data=val_generator,
        epochs=25,
        callbacks=callbacks_bf_ft
    )

    mlflow.log_param("phase_1_model_params", history.params)

    mlflow.log_metrics({
        "phase1_final_acc": history.history["accuracy"][-1],
        "phase1_final_val_acc": history.history["val_accuracy"][-1],
        "phase1_final_loss": history.history["loss"][-1],
        "phase1_final_val_loss": history.history["val_loss"][-1]
    })

    # ---- Modèle phase 1 ----
    mlflow.keras.log_model(model, name="Image_Classifier_bf_FT")
    # Log manuel des courbes
    import matplotlib.pyplot as plt

    # Loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title("Loss Phase 1")
    plt.savefig("loss_phase1.png")
    mlflow.log_artifact("loss_phase1.png")

    # Accuracy
    plt.figure()
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title("Accuracy Phase 1")
    plt.savefig("acc_phase1.png")
    mlflow.log_artifact("acc_phase1.png")

    from tensorflow.keras.models import load_model

    model = load_model("drive/MyDrive/Model/training_checkpoint_bf_ft.keras")
    test_loss, test_acc=model.evaluate(test_generator)

    mlflow.log_metrics({
        "test_acc":test_acc,
        "test_loss":test_loss
    })

    from sklearn.metrics import confusion_matrix
    import matplotlib.pyplot as plt
    import seaborn as sns

    y_pred_probs = model.predict(test_generator)
    y_pred = y_pred_probs.argmax(axis=1)

    from sklearn.metrics import classification_report, confusion_matrix
    report = classification_report(y_test, y_pred, target_names=selected_classes)
    with open("classification_report.txt", "w") as f:
      f.write(report)

    # Log MLflow
    mlflow.log_artifact("classification_report.txt")



    cm = confusion_matrix(y_test, y_pred)

    plt.figure(figsize=(10, 8))
    sns.heatmap(
    cm,
    xticklabels=selected_classes,
    yticklabels=selected_classes,
    cmap="Blues",
    annot=False
    )
    plt.title("Confusion Matrix")
    plt.ylabel("True label")
    plt.xlabel("Predicted label")

    plt.savefig("confusion_matrix.png", bbox_inches="tight")
    plt.close()

    mlflow.log_artifact("confusion_matrix.png")



2025/12/13 15:27:10 WARNING mlflow.keras.autologging: Unrecognized dataset type <class 'keras.src.legacy.preprocessing.image.NumpyArrayIterator'>. Dataset logging skipped.
2025/12/13 15:27:10 WARNING mlflow.keras.autologging: Unrecognized dataset type <class 'keras.src.legacy.preprocessing.image.NumpyArrayIterator'>. Dataset logging skipped.
/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/25
422/422 ━━━━━━━━━━━━━━━━━━━━ 0s 171ms/step - accuracy: 0.1113 - loss: 3.4727
Epoch 1: saving model to drive/MyDrive/Model/training_checkpoint_bf_ft.keras
422/422 ━━━━━━━━━━━━━━━━━━━━ 96s 197ms/step - accuracy: 0.1115 - loss: 3.4716 - val_accuracy: 0.5913 - val_loss: 1.8391 - learning_rate: 1.0000e-04
Epoch 2/25
422/422 ━━━━━━━━━━━━━━━━━━━━ 0s 163ms/step - accuracy: 0.3956 - loss: 2.2216
Epoch 2: saving model to drive/MyDrive/Model/training_checkpoint_bf_ft.keras
422/422 ━━━━━━━━━━━━━━━━━━━━ 122s 168ms/step - accuracy: 0.3957 - loss: 2.2213 - val_accuracy: 0.6627 - val_loss: 1.2866 - learning_rate: 1.0000e-04
Epoch 3/25
422/422 ━━━━━━━━━━━━━━━━━━━━ 0s 161ms/step - accuracy: 0.4900 - loss: 1.8200
Epoch 3: saving model to drive/MyDrive/Model/training_checkpoint_bf_ft.keras
422/422 ━━━━━━━━━━━━━━━━━━━━ 70s 166ms/step - accuracy: 0.4900 - loss: 1.8198 - val_accuracy: 0.6913 - val_loss: 1.1274 - learning_rate: 1.0000e-04
Epoch 4/25
422/422 ━━━━━━━━━━━━━━━━━━━━ 0s 161ms/step - accu

In [ ]:
from tensorflow.keras.models import load_model

model = load_model("drive/MyDrive/Model/training_checkpoint_bf_ft.keras")
test_loss, test_acc=model.evaluate(test_generator)


94/94 ━━━━━━━━━━━━━━━━━━━━ 11s 62ms/step - accuracy: 0.7947 - loss: 0.6935


In [ ]:
y_pred_probs = model.predict(test_generator)
y_pred = y_pred_probs.argmax(axis=1)

from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_test, y_pred, target_names=selected_classes))


94/94 ━━━━━━━━━━━━━━━━━━━━ 10s 68ms/step
              precision    recall  f1-score   support

      bottle       0.87      0.89      0.88       100
        bowl       0.64      0.59      0.61       100
         can       0.77      0.79      0.78       100
         cup       0.82      0.80      0.81       100
       plate       0.78      0.74      0.76       100
       apple       0.84      0.92      0.88       100
    mushroom       0.89      0.96      0.92       100
      orange       0.80      0.90      0.85       100
        pear       0.79      0.80      0.80       100
sweet_pepper       0.84      0.69      0.76       100
       clock       0.83      0.81      0.82       100
    keyboard       0.92      0.87      0.89       100
        lamp       0.71      0.77      0.74       100
   telephone       0.80      0.70      0.75       100
  television       0.78      0.86      0.82       100
         bed       0.77      0.72      0.74       100
       chair       0.85      0.82      0

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam

# Charger le modèle déjà entraîné du top classifier
model = load_model("drive/MyDrive/Model/training_checkpoint_bf_ft.keras")

# Dégeler les 90 dernières couches
fine_tune_at = len(model.layers) - 90
for layer in model.layers[:fine_tune_at]:
    layer.trainable = False
for layer in model.layers[fine_tune_at:]:
    layer.trainable = True
    # Ajouter régularisation L2 légère
    if hasattr(layer, 'kernel_regularizer'):
        layer.kernel_regularizer = l2(1e-5)

# Compiler avec un learning rate très faible
model.compile(
    optimizer=Adam(1e-4),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Entraîner
history_finetune = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=25,
    callbacks=callbacks_ft
)


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/25
422/422 ━━━━━━━━━━━━━━━━━━━━ 0s 222ms/step - accuracy: 0.6149 - loss: 1.4205

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()



Epoch 1: saving model to drive/MyDrive/Model/training_checkpoint_ft.keras
422/422 ━━━━━━━━━━━━━━━━━━━━ 146s 253ms/step - accuracy: 0.6150 - loss: 1.4201 - val_accuracy: 0.7273 - val_loss: 1.0127 - learning_rate: 1.0000e-04
Epoch 2/25
422/422 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step - accuracy: 0.7367 - loss: 0.9566
Epoch 2: saving model to drive/MyDrive/Model/training_checkpoint_ft.keras
422/422 ━━━━━━━━━━━━━━━━━━━━ 73s 173ms/step - accuracy: 0.7367 - loss: 0.9566 - val_accuracy: 0.7580 - val_loss: 0.8331 - learning_rate: 1.0000e-04
Epoch 3/25
422/422 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step - accuracy: 0.7863 - loss: 0.8151
Epoch 3: saving model to drive/MyDrive/Model/training_checkpoint_ft.keras
422/422 ━━━━━━━━━━━━━━━━━━━━ 75s 177ms/step - accuracy: 0.7863 - loss: 0.8151 - val_accuracy: 0.7840 - val_loss: 0.8068 - learning_rate: 1.0000e-04
Epoch 4/25
422/422 ━━━━━━━━━━━━━━━━━━━━ 0s 168ms/step - accuracy: 0.8077 - loss: 0.7157
Epoch 4: saving model to drive/MyDrive/Model/training_checkpoint_ft.

In [ ]:
from tensorflow.keras.models import load_model

model = load_model("drive/MyDrive/Model/training_checkpoint_ft.keras")
test_loss, test_acc=model.evaluate(test_generator)


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


94/94 ━━━━━━━━━━━━━━━━━━━━ 19s 138ms/step - accuracy: 0.8631 - loss: 0.5270


In [ ]:
y_pred_probs = model.predict(test_generator)
y_pred = y_pred_probs.argmax(axis=1)

from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_test, y_pred, target_names=selected_classes))

In [ ]:
import matplotlib.pyplot as plt

# Récupération des métriques pour la phase 1
loss1 = history.history['loss']
val_loss1 = history.history['val_loss']
acc1 = history.history['accuracy']
val_acc1 = history.history['val_accuracy']

# Récupération des métriques pour la phase 2
loss2 = history_finetune.history['loss']
val_loss2 = history_finetune.history['val_loss']
acc2 = history_finetune.history['accuracy']
val_acc2 = history_finetune.history['val_accuracy']

# Concaténation pour faire une seule courbe continue
loss = loss1 + loss2
val_loss = val_loss1 + val_loss2
acc = acc1 + acc2
val_acc = val_acc1 + val_acc2

# Nombre total d'époques
epochs_total = range(1, len(loss) + 1)

# ---- PLOT LOSS ----
plt.figure(figsize=(10,5))
plt.plot(epochs_total, loss, label='Training Loss')
plt.plot(epochs_total, val_loss, label='Validation Loss')
plt.axvline(x=len(loss1), color='gray', linestyle='--', label="Début fine-tuning")
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training & Validation Loss')
plt.legend()
plt.grid(True)
plt.savefig("drive/MyDrive/plots/training_validation_loss_2.png", dpi=300)
plt.show()

# ---- PLOT ACCURACY ----
plt.figure(figsize=(10,5))
plt.plot(epochs_total, acc, label='Training Accuracy')
plt.plot(epochs_total, val_acc, label='Validation Accuracy')
plt.axvline(x=len(acc1), color='gray', linestyle='--', label="Début fine-tuning")
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training & Validation Accuracy')
plt.legend()
plt.grid(True)
plt.savefig("drive/MyDrive/plots/training_validation_accuracy_2.png", dpi=300)
plt.show()


NameError: name 'history' is not defined